# Classifier comparison

A comparison of a several classifiers on CICIDS2017 webattacks dataset.

Sources:

* CICIDS2017: https://www.unb.ca/cic/datasets/ids-2017.html
* Scikit-learn demo: https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html
* Overview of classification metrics: http://www.machinelearning.ru/wiki/images/d/de/Voron-ML-Quality-slides.pdf

## Reading and preparing data

Read undersampled (balanced) and preprocessed data.

In [1]:
import pandas as pd
import numpy as np

In [2]:
!wget https://github.com/fisher85/ml-cybersecurity/blob/master/python-web-attack-detection/datasets/web_attacks_balanced.zip?raw=true -O web_attacks_balanced.zip
!unzip -u web_attacks_balanced.zip
df = pd.read_csv('web_attacks_balanced.csv')

The "Label" column is encoded as follows: "BENIGN" = 0, attack = 1.

In [3]:
df['Label'] = df['Label'].apply(lambda x: 0 if x == 'BENIGN' else 1)
y = df['Label'].values

Select the features.

In [4]:
webattack_features = ['Average Packet Size', 'Flow Bytes/s', 'Max Packet Length', 'Packet Length Mean', 
                      'Fwd Packet Length Mean', 'Subflow Fwd Bytes', 'Fwd IAT Min', 'Avg Fwd Segment Size',
                      'Total Length of Fwd Packets', 'Fwd IAT Std', 'Fwd Packet Length Max', 'Flow IAT Mean',
                      'Fwd Header Length', 'Flow Duration', 'Flow Packets/s', 'Fwd IAT Mean',
                      'Fwd IAT Total', 'Fwd Packets/s', 'Flow IAT Std', 'Fwd IAT Max']

In [5]:
X = df[webattack_features]
print(X.shape, y.shape)

(7267, 20) (7267,)


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

{0: 3556, 1: 1530}

In [7]:
X.head()

,Average Packet Size,Flow Bytes/s,Max Packet Length,Packet Length Mean,Fwd Packet Length Mean,Subflow Fwd Bytes,Fwd IAT Min,Avg Fwd Segment Size,Total Length of Fwd Packets,Fwd IAT Std,Fwd Packet Length Max,Flow IAT Mean,Fwd Header Length,Flow Duration,Flow Packets/s,Fwd IAT Mean,Fwd IAT Total,Fwd Packets/s,Flow IAT Std,Fwd IAT Max
0,80.75,3.689366e+03,103.0,64.6,39.0,78.0,4.0,39.0,78.0,0.0,39.0,25659.333330,64.0,76978.0,51.962898,4.0,4.0,25.981449,44436.340820,4.0
1,80.75,3.635433e+03,103.0,64.6,39.0,78.0,3.0,39.0,78.0,0.0,39.0,26040.000000,64.0,78120.0,51.203277,3.0,3.0,25.601638,45096.540850,3.0
2,63.50,1.082927e+06,79.0,50.8,32.0,64.0,37.0,32.0,64.0,0.0,32.0,68.333333,64.0,205.0,19512.195120,37.0,37.0,9756.097561,83.530434,37.0
3,94.25,1.928994e+06,112.0,75.4,51.0,102.0,3.0,51.0,102.0,0.0,51.0,56.333333,64.0,169.0,23668.639050,3.0,3.0,11834.319530,91.511384,3.0
4,94.25,1.097643e+06,112.0,75.4,51.0,102.0,4.0,51.0,102.0,0.0,51.0,99.000000,64.0,297.0,13468.013470,4.0,4.0,6734.006734,90.415707,4.0


## Classifier comparison

The operation may take a long time, 3-5 minutes depending on the computer performance.

In [8]:
import time
import warnings
warnings.filterwarnings("ignore")

from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier

models = []
models.append(('KNN', KNeighborsClassifier()))
models.append(('SVM', SVC(gamma='auto')))
models.append(('CART', DecisionTreeClassifier(max_depth=5)))
models.append(('RF', RandomForestClassifier(max_depth=5, n_estimators=5, max_features=3)))    
models.append(('ABoost', AdaBoostClassifier()))
models.append(('LR', LogisticRegression(solver='lbfgs', max_iter=200)))
models.append(('NB', GaussianNB()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('QDA', QuadraticDiscriminantAnalysis()))
models.append(('MLP', MLPClassifier()))

print('Model\tAcc\tPr\tRecall\tF1\tExecution')
      
for name, model in models:
    start_time = time.time()
    kfold = model_selection.KFold(n_splits=5, random_state=24)    

    accuracy = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy').mean()
    precision = cross_val_score(model, X_train, y_train, cv=kfold, scoring='precision').mean()
    recall = cross_val_score(model, X_train, y_train, cv=kfold, scoring='recall').mean()
    f1_score = cross_val_score(model, X, y, cv=kfold, scoring='f1_weighted').mean()
    
    delta = time.time() - start_time
    print('{}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.2f} secs'.format(name, accuracy, precision, recall, f1_score, delta))

Model	Acc	Pr	Recall	F1	Execution
KNN	0.966	0.936	0.951	0.969	1.02 secs
SVM	0.703	0.618	0.032	0.603	46.60 secs
CART	0.965	0.928	0.957	0.964	0.54 secs
RF	0.968	0.959	0.911	0.963	0.42 secs
ABoost	0.974	0.966	0.947	0.971	8.02 secs
LR	0.956	0.970	0.880	0.947	3.14 secs
NB	0.735	0.532	0.990	0.775	0.20 secs
LDA	0.933	0.909	0.862	0.940	0.65 secs
QDA	0.866	0.706	0.656	0.866	0.21 secs
MLP	0.942	0.874	0.920	0.964	77.35 secs
